In [6]:
# Once you get a problem to simulate and define steps followed, you first identify the variables you will need in your simulation
# decide which variables have constant values and those that will need to be generated randomly
# identify the processes that need to be carried out in your environment
# import necessary variables for your simulation

# create and initialize the environment
# create functions for your defined processes

In [ ]:
import simpy
import random

RANDOM_SEED = 42
NUM_MACHINES = 2  # 2 washing machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes

In [7]:
class Carwash(object):
    """A carwash has a limited number of machines (``NUM_MACHINES``) to
    clean cars in parallel.

    Cars have to request one of the machines. When they got one, they
    can start the washing processes and wait for it to finish (which
    takes ``washtime`` minutes).

    """
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime # the washtime is a defined number of minutes (5 ,minutes)

        # define a function for the washing process
    def wash(self, car):
        """The washing processes. It takes a ``car`` processes and tries
        to clean it."""
        yield self.env.timeout(WASHTIME) # wash a car after every 5 minutes
        # a breakdown of the execution of this code below which gives a random precision of the car washing 
        # "%d%%" % (random.randint(50,90)) - random percentage between 50 and 90
        # "%s's" (car) - the values obtained here are from the variable car
        print("Carwash removed %d%% of %s's dirt." % 
              (random.randint(50, 99), car))

In [8]:
def car(env, name, cw):
    """The car process (each car has a ``name``) arrives at the carwash
    (``cw``) and requests a cleaning machine.

    It then starts the washing process, waits for it to finish and
    leaves to never come back ...

    """
    # %.2f - 2 significant figures
    print('%s arrives at the carwash at %.2f.' % (name, env.now)) # first car arrives at car wash
    with cw.machine.request() as request: # a car named 'name' requests for a machine and gets whichever is available, and after it's washed, the machine is declared available again 
        yield request

        print('%s enters the carwash at %.2f.' % (name, env.now)) # first car enters the machine 
        yield env.process(cw.wash(name)) # the washing process is executed for this particular car called 'name'

        print('%s leaves the carwash at %.2f.' % (name, env.now)) # print out the name of car and time when the car completed the washing process

In [9]:
def setup(env, num_machines, washtime, t_inter):
    """Create a carwash, a number of initial cars and keep creating cars
    approx. every ``t_inter`` minutes."""
    # Create an instance of the carwash
    carwash = Carwash(env, num_machines, washtime)

    # Create 4 initial cars that were there when the car wash got opened
    for i in range(4):
        env.process(car(env, 'Car %d' % i, carwash)) # this is the code that auto-increments the name of the car for the first 4

    # Create more cars while the simulation is running
    while True: # while the simulation is running
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1 # this is the code that auto-increments the name of the car for the rest of hte cars generated for the simulation as it runs
        env.process(car(env, 'Car %d' % i, carwash))

In [10]:
# Setup and start the simulation
print('Carwash')
print('Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)')
random.seed(RANDOM_SEED)  # This helps reproducing the results, initializing an RNG

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

Carwash
Check out http://youtu.be/fXXmeP9TvBg while simulating ... ;-)


<Process(setup) object at 0x2374337e700>

In [11]:
env.run(until=SIM_TIME)

Car 0 arrives at the carwash at 0.00.
Car 1 arrives at the carwash at 0.00.
Car 2 arrives at the carwash at 0.00.
Car 3 arrives at the carwash at 0.00.
Car 0 enters the carwash at 0.00.
Car 1 enters the carwash at 0.00.
Car 4 arrives at the carwash at 5.00.
Carwash removed 97% of Car 0's dirt.
Carwash removed 67% of Car 1's dirt.
Car 0 leaves the carwash at 5.00.
Car 1 leaves the carwash at 5.00.
Car 2 enters the carwash at 5.00.
Car 3 enters the carwash at 5.00.
Car 5 arrives at the carwash at 10.00.
Carwash removed 64% of Car 2's dirt.
Carwash removed 58% of Car 3's dirt.
Car 2 leaves the carwash at 10.00.
Car 3 leaves the carwash at 10.00.
Car 4 enters the carwash at 10.00.
Car 5 enters the carwash at 10.00.
Carwash removed 97% of Car 4's dirt.
Carwash removed 56% of Car 5's dirt.
Car 4 leaves the carwash at 15.00.
Car 5 leaves the carwash at 15.00.
Car 6 arrives at the carwash at 16.00.
Car 6 enters the carwash at 16.00.
